In [1]:
import time
import random
from functools import wraps
import pandas as pd
import datetime as dt


def print_args(f):
    @wraps(f)
    def func(*args, **kwargs):
        print(f"function is called with: {args}, {kwargs}")
        result = f(*args, **kwargs)
        return result
    return func


def loggg(show_time=True, show_name=True):
    def stopwatch(f):
        @wraps(f)
        def func(*args, **kwargs):
            tic = time.time()
            result = f(*args, **kwargs)
            log_text = "call"
            if show_name:
                log_text = f"{log_text} {f.__name__}"
            if show_time:
                log_text = f"{log_text} time:{time.time() - tic}"
            print(log_text)
            return result
        return func
    return stopwatch


def loggg_2(func_in=None, *, show_time=True, show_name=True):
    def stopwatch(f):
        @wraps(f)
        def func(*args, **kwargs):
            tic = time.time()
            result = f(*args, **kwargs)
            log_text = "call"
            if show_name:
                log_text = f"{log_text} {f.__name__}"
            if show_time:
                log_text = f"{log_text} time:{time.time() - tic}"
            print(log_text)
            return result
        return func
    if func_in is None:
        return stopwatch
    else:
        return stopwatch(func_in)


@print_args
@loggg_2(show_time=True, show_name=True)
def sleep_random(s=1):
    """This function sleeps for at least 's' seconds."""
    time.sleep(s + random.random())
    return "Done"


@print_args
@loggg_2
def sleep_random_2(s=1):
    """This function sleeps for at least 's' seconds."""
    time.sleep(s + random.random())
    return "Done"

In [2]:
sleep_random(s=1)

function is called with: (), {'s': 1}
call sleep_random time:1.367227554321289


'Done'

In [3]:
sleep_random_2(s=1)

function is called with: (), {'s': 1}
call sleep_random_2 time:1.5446536540985107


'Done'

In [4]:
help(sleep_random)

Help on function sleep_random in module __main__:

sleep_random(s=1)
    This function sleeps for at least 's' seconds.



In [5]:
def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"just ran step {func.__name__} shape={result.shape} took {time_taken}s")
        return result
    return wrapper

In [6]:
df = pd.read_csv('https://calmcode.io/datasets/bigmac.csv')

@log_step
def start_pipeline(dataf):
    return dataf.copy()

@log_step
def set_dtypes(dataf):
    return (dataf
            .assign(date=lambda d: pd.to_datetime(d['date']))
            .sort_values(['currency_code', 'date']))

@log_step
def remove_outliers(dataf, min_row_country=32):
    countries = (dataf
                .groupby('currency_code')
                .agg(n=('name', 'count'))
                .loc[lambda d: d['n'] >= min_row_country]
                .index)
    return (dataf
            .loc[lambda d: d['currency_code'].isin(countries)])

df_new = (df
  .pipe(start_pipeline)
  .pipe(set_dtypes)
  .pipe(remove_outliers, min_row_country=20))

just ran step start_pipeline shape=(1330, 6) took 0:00:00.000337s
just ran step set_dtypes shape=(1330, 6) took 0:00:00.020967s
just ran step remove_outliers shape=(1248, 6) took 0:00:00.020549s


In [9]:
from retry import retry

import logging
logging.basicConfig()

@retry(ValueError, tries=5, delay=0.5)
def randomly_fails(p=0.5):
    if random.random() < p:
        raise ValueError("no bueno!")
    return "Done!"

randomly_fails()

'Done!'